In [37]:
import pandas as pd
import re
import nltk
import re
from stopwordsiso import stopwords as stopwords_iso
from nltk.corpus import stopwords as stopwords_nltk
# from langdetect import detect
import stanza
from tqdm import tqdm
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException
from lingua import Language, LanguageDetectorBuilder
# from transformers import pipeline

In [38]:
# PROCESSORS
languages = [Language.ENGLISH, Language.TAGALOG]
detector = LanguageDetectorBuilder.from_languages(*languages).build()
stanza.download('en')
# TAGALOG TOKENIZER FOR SENTENCES THAT ARE DOMINANT IN TAGALOG WORDS (used to segment sentences into a bag of words)
nlp = stanza.Pipeline('en', processors="tokenize,lemma",use_gpu=True, treebank='ESLSpok')
# STOP WORDS for both languages
nltk.download('stopwords')
stopwords_all = stopwords_nltk.words('english') + list(stopwords_iso('tl')) + ["im", "i'm", "i am"]

# translator
# translator = GoogleTranslator(source='auto', target='en')

# pipeline for bert transformer
# bert_pipe_tiai = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

2025-02-24 13:43:30 INFO: Downloaded file to C:\Users\rainn\stanza_resources\resources.json
2025-02-24 13:43:30 INFO: Downloading default packages for language: en (English) ...
2025-02-24 13:43:31 INFO: File exists: C:\Users\rainn\stanza_resources\en\default.zip
2025-02-24 13:43:34 INFO: Finished downloading models and saved to C:\Users\rainn\stanza_resources
2025-02-24 13:43:34 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-02-24 13:43:34 INFO: Downloaded file to C:\Users\rainn\stanza_resources\resources.json
2025-02-24 13:43:34 WARNING: Language en package default expects mwt, which has been added
2025-02-24 13:43:34 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| lemma     | combined_nochar

In [39]:
tagalog_contractions = {
    'sya': 'siya',
    'bat': 'bakit',
    'ikay': 'ikaw ay',
    'sakin': 'sa akin',
    'sayo': 'sa iyo',
    'samin': 'sa amin',
    'withraw':'withdraw',
    'satin': 'sa atin',
    'niyo':'ninyo',
    'nyo': 'ninyo',
    'nang': 'na ang',
    'tas': 'tapos',
    'meron': 'mayroon',
    'penge': 'pahingi',
    'di' : 'hindi',
    'pls': 'please',
    'sna': 'sana',
    'gomana': 'gumana',
    'dili': 'hindi',
    'hoi': 'hey',
    'dko': 'hindi ko',
    'nmn':'',
    'naman':'',
    'nman':'',
    'kaka':'',
    'bkt': 'bakit',
    'prin': 'parin',
    'nkaka':'',
    'pla':'pala',
    '2fa': 'authentication',
    'transac': 'transaction',
    'app': 'application',
    'nah':'no',
    'pesos':'peso',
    'php':'peso',
    'pera':'money',
    'apps':'application',
    'bwesit':'bwisit',
    'bwiset':'bwisit',
    'tngina':'putangina',
    'tangina':'putangina',
    'tnginaka':'putangina',
    'kingina':'putangina',
    'mbgal':'bagal',
    'mabagal':'bagal',
    'gcahs':'gcash',
    'gagu':'gago',
    'bubu':'bobo',
    'ogag':'gago',
    'obob':'bobo',
    'tgal':'matagal',
    'tagal':'matagal',
    'mtagal':'matagal',
    'accnt':'account',
    'recieve':'receive',
    'recieved':'receive',
    'ive':'have',
    'ayosin':'ayusin',
    'q': 'ko',
    'xa':'siya',
    'ug':'iyon',
    'yung':'iyon',
    'yong':'iyon',
    'imong':'iyon',
    'hindot':'putangina',
    'lage':'palagi',
    'laging':'palagi',
    'lagi':'palagi',
    'palaging':'palagi',
    'bkit':'bakit',
    'lyk':'like',
    'dis':'this',
    'nko':'nako',
    'de':'hindi',
    'dl':'download',
    'cp':'phone',
    'cellphone':'phone',
    'ai':'artificial intelligence',
    'nd':'',
    'ndi':'hindi',
    'hndi':'hindi',
    'andami':'mada',
    'pinag':'',
    'kapag':'',
    'naka':'',
    'nakaka':'',
    'nkka':'',
    'dabest':'best', 
    'pakyu':'putangina',
    'ampanget':'pangit',
    'panget':'pangit',
    'hina':'mahina',
    
}
negation_words = [
    "not", "no", "never", "none", "nothing", "nowhere", "neither", "nor", "without",
    "don't", "do not", "doesn't", "does not", "didn't", "did not",
    "can't", "cannot", "couldn't", "could not", "won't", "will not",
    "wouldn't", "would not", "shouldn't", "should not",
    "isn't", "is not", "aren't", "are not", "wasn't", "was not", "weren't", "were not",
    "hasn't", "has not", "haven't", "have not", "hadn't", "had not",
    "cant", "couldnt", "doesnt", "wouldnt", "dont", "wasnt", "werent", "hadnt", "arent", "wont", "shouldnt", "havent", 
    
    "hindi", "hnd", "ndi", "wala", "walang",
    "ayaw", "wag", "huwag", "hnd pa", "ndi pa", "di pa", "wala pa",
    "huwag na", "hnd pa", "hindi pa", "ayoko", "di ako", "wala nang",
    "di mo", "d mo", "di siya", "d siya", "di sila",
    "hindi rin", "hnd rin", "hindi naman", "wala ito", "wala akong",
]


In [40]:
# PLAYGROUND
    # stopwords_nltk.words('english')
# print(bert_pipe_tiai("Putanginaaaa mong gcash ka andami konang Pera nagloko pa madaya yang putanginang yan Wag nyo to i, install putangggina"))
# print(bert_pipe_dost("Putanginaaaa mong gcash ka andami konang Pera nagloko pa madaya yang putanginang yan Wag nyo to i, install putangggina"))
# print(bert_pipe_dost("this is not good gchash"))

In [41]:
# LOAD DF ONLY GET IMPORTANT COLUMNS
load_list = ['2020-','2021-','2022-', '2023-']
def load_df(load):
    # removes words that are less than 3 letters, no letters, stopwords, contractions, unnecessary columns, and null values
    data = pd.read_csv('GCASH_REVIEWS.csv')
    data = data[data['review_text'].str.contains(r'[a-zA-Z]', na=False)]
    data = data[data['review_text'].str.len() > 3]
    data = data[~data['review_text'].isin(tagalog_contractions)]
    data = data[~data['review_text'].isin(stopwords_all)]
    data = data.drop(columns=['author_app_version', 'author_id', 'author_name'])
    data = data.dropna()
    original = data.copy()
    # FILTER BY YEAR just change 20XX-
    data = data[data['review_datetime_utc'].str.startswith(load)]
    return data, original

# df, df_original = load_df()

In [58]:
# 'text' is one row of string of words for every row in df['review_text']
contrast_words = ["pero", "ngunit", "subalit", "kahit na", "pero kasi", "but", "however", "although", "though", "while", "kaso"]
transac_words = {"log in": "login", "log out": "login", "sign in": "signin", "sign out": "signout", "g cash": "gcash", "g-cash":"gcash"}
def handleContrast(text: str) -> str:
    for contrast in contrast_words:\
        text = re.sub(rf"\b{re.escape(contrast)}\b", " contrastlabel ", text)
        # text = text.replace(" "+contrast+" ", " contrastlabel ")
    return text

def handleNegation(text: str) -> str:
    for neg in negation_words:
        text = re.sub(rf"\b{re.escape(neg)}\b", " negativelabel ", text)
        # text = text.replace(" "+neg+" ", " negativelabel ")
    return text
def simplifyTransacWords(text: str) -> str:
    for tw in transac_words:
        text =  re.sub(rf"\b{re.escape(tw)}\b", " "+transac_words[tw]+" ", text)
        # text = text.replace(" "+tw+" ", " "+transac_words[tw]+" ")
    return text
# label_obj = {"Very Negative" : 'verynegativelabel', "Negative": 'negativelabel', "Neutral": 'neutrallabel', "Positive": 'positivelabel', "Very Positive": 'verypositivelabel'}
def clean_text(text: str):
    
    # handles negation identification and adds negative label as placeholder to not be removed in stopwords
    text = text.lower()
    text_negation_handled = handleNegation(text)
    
    # filters text, remove special characters and emoji
    text = re.sub(r"[^a-zA-Z0-9]", " ", text_negation_handled)
    
    text_contrast_handled = handleContrast(text)
    text = simplifyTransacWords(text_contrast_handled)
    
    # turn tagalogs into their non contraction (expanded) form if available (e.g satin -> sa atin)
    words = text.split()
    non_contracted_words = [(tagalog_contractions[w] if w in tagalog_contractions else w) for w in words]

    # exclude stopwords for english and tagalog (e.g ang, may, ako, myself, is, the)
    no_stop_words = [ncw for ncw in non_contracted_words if ncw not in stopwords_all]

    
    filtered_words = ' '.join(no_stop_words)
    filtered_words = filtered_words.lower()
    
    # do lemmatizing + base form to identify words of context (e.g running -> run, ran -> run) (english words only)
    doc = nlp(filtered_words)
    principal_words = []
    for sentence in doc.sentences:
        for word in sentence.words:
            w = word.lemma if word.lemma else word.text
            
            # remove standalone numbers, as well gibberish words
            if len(w) < 2 or w.isdigit(): 
                continue
            # if detect(w) in ['en', 'tl']: 
            principal_words.append(w)
    
    # printouts
    
    # print("regex applied: ", text)
    # print("negation handled: ", text_negation_handled)
    # print("contrast handled: ", text_contrast_handled)

    # print("noncontractedwords ", non_contracted_words)
    # print("no stop words", no_stop_words)
    # print("filtered_words ", filtered_words)
    
    # then return an array of words, remove low effort/generic reviews 
    return principal_words if len(principal_words) > 2 else None
            
# clean_text("Ok 🎉 sya gamitin ung SSS .but sometimes may difficulties,...but worthit ..na intact parin yung laman ng gcash? 😂")
    
    


In [43]:
simplifyTransacWords("g-cash g cash sucks")

' gcash   gcash  sucks'

In [55]:
clean_text("dont know why but g-cash g cash sucks, cant login or sign-in?")

regex applied:   negativelabel  know why  contrastlabel   gcash   gcash  sucks   negativelabel  login or  signin  
negation handled:   negativelabel  know why but g-cash g cash sucks,  negativelabel  login or sign-in?
contrast handled:   negativelabel  know why  contrastlabel  g cash g cash sucks   negativelabel  login or sign in 
noncontractedwords  ['negativelabel', 'know', 'why', 'contrastlabel', 'gcash', 'gcash', 'sucks', 'negativelabel', 'login', 'or', 'signin']
no stop words ['negativelabel', 'know', 'contrastlabel', 'gcash', 'gcash', 'sucks', 'negativelabel', 'login', 'signin']
filtered_words  negativelabel know contrastlabel gcash gcash sucks negativelabel login signin


['negativelabel',
 'know',
 'contrastlabel',
 'gcash',
 'gcash',
 'suck',
 'negativelabel',
 'login',
 'signin']

In [59]:
for load in load_list:
    print('FOR: ', load)
    df, _ = load_df(load)
    df['review_text_formatted'] = [clean_text(text) for text in tqdm(df['review_text'])]
    df.to_csv('with_contrast/gcash-{}formatted.csv'.format(load))
    print()

FOR:  2020-


100%|██████████| 72698/72698 [12:45<00:00, 94.93it/s] 



FOR:  2021-


100%|██████████| 144343/144343 [24:04<00:00, 99.90it/s] 



FOR:  2022-


100%|██████████| 157895/157895 [30:30<00:00, 86.25it/s] 



FOR:  2023-


100%|██████████| 133050/133050 [31:16<00:00, 70.89it/s] 


In [ ]:
# df_sample = df.sample(100)
# df_sample['review_text_formatted'] = [clean_text(text) for text in tqdm(df_sample['review_text'])]

In [9]:
print('hel;lo{}'.format(load_list[0]))

hel;lo2020-
